In [1]:
%load_ext autoreload
%autoreload 2

import lightning.pytorch as pl
from lightning.pytorch.loggers import WandbLogger

from model import TPALSTM
import pandas as pd
from dataset import ElectricityDataModule

In [2]:
data_df = pd.read_csv('data/clean_and_merge_data.csv', index_col=0)

In [3]:
data_splits = {
    "train": 0.7,
    "val": 0.1,
    "predict": 0.2
}

elec_dm = ElectricityDataModule(
    dataset_splits=data_splits,
    batch_size=64,
    window_size=24
)

model = TPALSTM(
    input_size=data_df.shape[1],
    output_horizon=1,
    hidden_size=25,
    obs_len=24,
    n_layers=1
)

wandb_logger = WandbLogger(
    name='tpa-lstm',
    save_dir='logs',
    project='Time-Series project',
    log_model=True
)

trainer = pl.Trainer(
    max_epochs=100,
    # accelerator=auto,
    logger=wandb_logger
)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: fraikinarchie (mva-data-challenge). Use `wandb login --relogin` to force relogin


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [4]:
trainer.fit(model, elec_dm)


  | Name      | Type                     | Params
-------------------------------------------------------
0 | hidden    | Linear                   | 1.9 K 
1 | relu      | ReLU                     | 0     
2 | lstm      | LSTM                     | 5.2 K 
3 | attention | TemporalPatternAttention | 3.1 K 
4 | linear    | Linear                   | 26    
5 | criterion | MSELoss                  | 0     
-------------------------------------------------------
10.2 K    Trainable params
0         Non-trainable params
10.2 K    Total params
0.041     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/Users/damso/opt/anaconda3/envs/venv/lib/python3.8/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/Users/damso/opt/anaconda3/envs/venv/lib/python3.8/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

/Users/damso/opt/anaconda3/envs/venv/lib/python3.8/site-packages/lightning/pytorch/trainer/call.py:54: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
